This code runs a basic digital quantum simulation of the time evolution of a single particle in 1-D space.
We follow the general framework introduced in the slides: 

1. How do we represent the position and momentum of the particle using qubits?

2. How do we initialize and measure the qubits? 

3. How do we simulate time evolution? 

4. How do we change between momentum and position bases? 

5. How do we implement diagonal unitaries? 

The first two are done naturally using the standard Qiskit operations. Time evolution can be computed using trotterization. The bases changes are done using QFT, since the fourier transform of a particle's state in the momentum basis gives us the state in the position basis. 

### Simulating Time Evolution
We will use the following Hamiltonian to simulate time evolution

$\hat{H} = \frac{\hat{p}^2}{2m} + V(\hat{x})$

which can be represented in our circuit using the unitary

$\hat{U} = \text{exp}(-i\hat{H}t / \hslash)$

Import the necessary components

In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import math

# importing Qiskit
from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.visualization import plot_histogram, plot_bloch_vector
from qiskit.quantum_info import Statevector

# import basic plot tools
from qiskit.visualization import plot_histogram

# Example 1: Gaussian Wave Function with No Potential

## Outline of Kitaev-Webb Algorithm 
To prepare the Gaussian wavefunction, we use the Kitaev-Webb Algorithm that is outlined as follows: 
![Outline of Kitaev-Webb](./Kitaev-Webb.png)

This algorithm can create the state of the periodic discrete Gaussian, which can be expressed recursively as 
$$
\newcommand{\bra}[1]{\left< #1 \right|}
\newcommand{\ket}[1]{\left| #1 \right>}
\newcommand{\bk}[2]{\left< #1 \middle| #2 \right>}
\newcommand{\bke}[3]{\left< #1 \middle| #2 \middle| #3 \right>}$$

$$\ket{\xi_{\mu, \sigma, N}} = \ket{\xi_{\frac{\mu}{2}, \frac{\sigma}{2}, N-1}} \otimes \cos(\alpha) \ket{0} + \ket{\xi_{\frac{\mu-1}{2}, \frac{\sigma}{2}, N-1}} \otimes \sin(\alpha)\ket{1} \tag{11}$$.


## Inputs

In [2]:
mu = 0
sigma = 1
k = 3
N = 3

mu_size = 1  # The number of qubits used to represent mu
sigma_size = 1  # The number of qubits used to represent sigma


## Step 1. Creating the initial state

In [3]:
def binary_approx(n, k):
    """
    
    Input:
        n: real number between 0 and 1
        k: the number of bits use to approximate n
        
    Output: 
        a: a list such that the i-th element is 1 or 0
    """
    a = []
    total = 0
    for i in range(1, k+1):
        bit = 0
        if (n > total + 1/2**i): 
            bit = 1
        else:
            bit = 0
        a.append(bit)
        total += bit/2**i
    return (a, total)

alpha_register = QuantumRegister(k, 'alpha')

mu_register = QuantumRegister(mu_size, 'mu')
sigma_register = QuantumRegister(sigma_size, 'sigma')
N_register = QuantumRegister(N, 'n')
res_register = QuantumRegister(N, 'result')

qc = QuantumCircuit(alpha_register, mu_register, sigma_register, N_register, res_register)
qc.draw()

alpha_0: 
          
 alpha_1: 
          
 alpha_2: 
          
    mu_0: 
          
 sigma_0: 
          
     n_0: 
          
     n_1: 
          
     n_2: 
          
result_0: 
          
result_1: 
          
result_2:

## Step 2. Compute $\alpha$

The angle $\alpha$ is defined as

$$\alpha = \cos^{-1}\left(\sqrt{\frac{f(\mu/2, \sigma/2)}{f(\mu, \sigma)}}\right). \tag{12}$$

The function $f$ is the normalization factor defined as follows: 

$$f(\mu, \sigma) = \sum_{n=-\infty}^{\infty} e^{-\frac{(n-\mu)^2}{\sigma^2}}. \tag{7}$$

To input the value $\alpha$ into our quantum circuit, we need to approximate this number using the number of qubits we have. Since $0 \leq \alpha \leq 2\pi$, we can approximate this number using k-digits of binary notation given by the formula $\frac{\alpha}{2\pi} \approx \sum_{i=1}^k \frac{a_i}{2^i}$ that gives us the digits we need to approximate $\alpha$ as $\alpha \approx \sum_{i=1}^k \pi\frac{a_i}{2^{i-1}}$. Here, $a_i$ takes the values either 0 or 1. This binary notation gives us a way to implement the rotation operator as a sequence of $k$ standard rotations as 
$$R(\alpha) \approx R(\pi/2^{k-1})^{\alpha_k}\cdots R(\pi/2^2)^{\alpha_2}R(\pi/2)^{\alpha_1}$$

The following block implements some routines to implement the functions needed to create $\alpha$. 

In [4]:
def f(mu, sigma, n):
    return np.sum(np.exp((-(np.arange(-n, n+1, 1) - mu)**2)/float(sigma**2)))

def angle(mu, sigma, n=1000):
    return np.arccos(math.sqrt(f(mu/2, sigma/2, n)/ f(mu, sigma, n)))

# Example code to verify correctness
alpha = angle(mu, sigma)/(np.pi*2)
precision = 6

(alpha_binary, alpha_approx) = binary_approx(alpha, precision)

print(alpha)
print(alpha_approx)
print(alpha_binary)

NameError: name 'g' is not defined

In [ ]:
def KWA(mu, sigma, k, N):
    """
    Kitaev-Webb Algorithm to construct the prepare the Gaussian wavefunction
    Parameters:
        - mu: mean
        - sigma: sd
        - k: 
    """

In [ ]:
qc = QuantumCircuit(4, 3)
# qc.x(3)

for qubit in range(4):
    qc.h(qubit)

qc.barrier() 

qc.cnot(0, 1)
qc.cnot(1, 2)
qc.cnot(2, 3)
qc.rz(math.pi/4, 3)
qc.cnot(2,3)
qc.cnot(1,2)
qc.cnot(0,1)

qc.barrier() 

for qubit in range(4):
    qc.h(qubit)

qc.draw('mpl')


In [ ]:
backend = Aer.get_backend('statevector_simulator') # Tell Qiskit how to simulate our circuit

state = execute(qc, backend).result().get_statevector()
print("Qubit State = " + str(state))


Notes: Fourier transform of the momentum operator is the position operator.